Obtaining data from Foursquare

In [5]:
import pandas as pd
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests

In [3]:
# Foursquare credentials

CLIENT_ID = 'PTKZ0AXVDVZ25SWDOE4LZ0INFJXD3YY4FBJOG3S5EWW2NVGB' # your Foursquare ID
CLIENT_SECRET = 'MC3FOKHMKJC1HZSJXXQN3WWS401RGX4EUYCNUQ3RKHCSDPJV' # your Foursquare Secret
VERSION = '20180323'
LIMIT = 500

In [38]:
# read in dataset

portzip= pd.read_csv("port_zip_cent.csv")
portzip.head()


,ZIP,PO_NAME,GRID_MEMB,SHAPE_area,SHAPE_len,latitude,longitude
0,97005,BEAVERTON,8,1.446614e+08,62126.24628,45.492081,-122.804066
1,97006,BEAVERTON,8,3.384048e+08,94176.92115,45.517390,-122.859185
2,97007,BEAVERTON,"7, 8",7.350534e+08,140315.74390,45.453584,-122.878375
3,97008,BEAVERTON,8,1.458592e+08,52474.89951,45.460845,-122.802951
4,97009,BORING,"7, 8",1.123944e+09,278545.75660,45.424101,-122.367481
5,97015,CLACKAMAS,"7, 8",4.920973e+08,140129.89630,45.410864,-122.506270
6,97024,FAIRVIEW,8,8.595267e+07,42520.87528,45.544780,-122.440990
7,97027,GLADSTONE,7,7.406243e+07,42503.79215,45.387256,-122.592319
8,97030,GRESHAM,8,2.119155e+08,78345.97631,45.509068,-122.430391
9,97034,LAKE OSWEGO,"7, 8",2.366281e+08,89966.19461,45.410763,-122.685022


In [40]:
zip_name = portzip.loc[25, 'PO_NAME']
portzip_latitude = portzip.loc[25, 'latitude'] 
portzip_longitude = portzip.loc[25, 'longitude'] # limit of number of venues returned by Foursquare API

#Set limit and radius
LIMIT = 100 
radius = 24140

# Set category ID's for Shoe Store, Outlet Mall, Outlet Store, Shoe Repair, Shopping Plaza, Shopping Mall (Respectively)
category_id = ['4bf58dd8d48988d107951735, 5744ccdfe4b0c0459246b4df, 52f2ab2ebcbc57f1066b8b35, 52f2ab2ebcbc57f1066b8b39, 5744ccdfe4b0c0459246b4dc, 4bf58dd8d48988d1fd941735']

In [53]:
# Send the GET request to Foursquare

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&id={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    portzip_latitude, 
    portzip_longitude,
    category_id,
    radius, 
    LIMIT) 

results = requests.get(url).json()

In [57]:
## So the code above worked.

#venues = results['response']
#venues = results['response']['groups'][0]['items']
#nearby_venues = json_normalize(venues)

<ipython-input-57-a6601c9b22c8>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


In [59]:
#nearby_venues.head(5)
#venues
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize('venues')
#dataframe.head()

KeyError: 'venues'

In [22]:
# Will need to refine our search. View map in meantime
locations = portzip[['latitude', 'longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]



[45.38725587, -122.59231902]

In [28]:
map = folium.Map(location=[45.38, -122.59], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=portzip['PO_NAME'][point]).add_to(map)
    #folium.MeasureControl().add_to(map) - Figure out how to add this.
map

